<a href="https://www.kaggle.com/code/ifeoluwafaromika/brain-tumor-detection-with-pytorch?scriptVersionId=164023755" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install torchsummary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shutil import copy, copytree
import cv2, os
from tqdm import tqdm
import math
from torchmetrics import Accuracy
from sklearn.metrics import confusion_matrix


import torch
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import functional
import PIL
import torchsummary
from torch import nn

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from Iacosh_Python.display import clear_output
clear_output()

In [ ]:
ROOT_DIR = '../input/brain-mri-images-for-brain-tumor-detection'

In [ ]:
!apt-get install tree
clear_output()
# create new folders
!mkdir data train test val data/yes data/no train/yes train/no test/yes test/no val/yes val/no 
!tree -d

In [ ]:
for category in os.listdir(ROOT_DIR):
    print(category)

In [ ]:
DATA_DIR = '/kaggle/working/data/'

for category in os.listdir(ROOT_DIR):
    if category != 'brain_tumor_dataset':
        for file in os.listdir(f'{ROOT_DIR}/{category}'):
            copy(f'{ROOT_DIR}/{category}/{file}', os.path.join(f'{DATA_DIR}/{category}', file))

In [ ]:
len(os.listdir(f'{ROOT_DIR}/no'))

In [ ]:
len(os.listdir(f'{DATA_DIR}/yes'))

In [ ]:
# Renaming file in the data directory

for category in os.listdir(DATA_DIR):
    for n, file in enumerate(os.listdir(f'{DATA_DIR}/{category}')):
        file_path = f'{DATA_DIR}/{category}/{file}'
        new_file_path = f'{DATA_DIR}/{category}/{n:03d}.jpg'  
#         print(file_path, new_file_path)
        os.rename(file_path, new_file_path)

## Visualizing to get identifying features

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(20, 8))

img1 = plt.imread(f'{DATA_DIR}/yes/092.jpg')
img2 = plt.imread(f'{DATA_DIR}/no/092.jpg')

img3 = plt.imread(f'{DATA_DIR}/yes/086.jpg')
img4 = plt.imread(f'{DATA_DIR}/no/086.jpg')

img5 = plt.imread(f'{DATA_DIR}/yes/042.jpg')
img6 = plt.imread(f'{DATA_DIR}/no/042.jpg')

img7 = plt.imread(f'{DATA_DIR}/yes/014.jpg')
img8 = plt.imread(f'{DATA_DIR}/no/014.jpg')

axes[0,0].imshow(img1, interpolation='nearest')
axes[0,1].imshow(img2, interpolation='nearest')
axes[0,0].set_title('YES')
axes[0,1].set_title('NO')

axes[1,0].imshow(img3, interpolation='nearest')
axes[1,1].imshow(img4, interpolation='nearest')

axes[2,0].imshow(img5, interpolation='nearest')
axes[2,1].imshow(img6, interpolation='nearest')

axes[3,0].imshow(img7, interpolation='nearest')
axes[3,1].imshow(img8, interpolation='nearest')

plt.tight_layout()
plt.show()

In [ ]:
# Moving files into train, test and val folders

for category in os.listdir(DATA_DIR):
    for n, file in enumerate(os.listdir(DATA_DIR + f'/{category}')):
        file_path = f'{DATA_DIR}/{category}/{file}'
        
        if n < 5:
            copy(file_path, f'test/{category}/{file}')
        elif n < 0.8 * len(os.listdir(DATA_DIR + f'/{category}')):
            copy(file_path, f'train/{category}/{file}')
        else:
            copy(file_path, f'val/{category}/{file}')

In [ ]:
train_transforms = transforms.Compose(
    [
#         transforms.RandomRotation(45),
#         transforms.RandomHorizontalFlip(),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
    ]
)

In [ ]:
TRAIN_DIR = '/kaggle/working/train/'
TRAIN_DATASET = ImageFolder(root=TRAIN_DIR, transform=train_transforms)

In [ ]:
sample_img_path = '../input/brain-mri-images-for-brain-tumor-detection/no/34 no.jpg'
image = PIL.Image.open(sample_img_path)

num_channels = functional.get_image_num_channels(image)

print('Number of channels: ', num_channels)

In [ ]:
count = 0

for file in os.listdir('/kaggle/working/val/yes'):
    sample_img_path = f'/kaggle/working/val/yes/{file}'
    image = PIL.Image.open(sample_img_path)

    num_channels = functional.get_image_num_channels(image)

#     print('Number of channels: ', num_channels)
    if num_channels > 1:
        count+=1
        
print(f"{count} images have 3 channels in the 'yes' category")

In [ ]:
print(f"Image size: {image.size}")

In [ ]:
class TumorClassifier(nn.Module):
    def __init__(self):
        super(TumorClassifier, self).__init__()
        self.relu = nn.ReLU()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.2)
        
#         self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
#         self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.dropout(self.pool1(self.relu(self.conv1(x))))
        x = self.dropout(self.pool2(self.relu(self.conv2(x))))
#         x = self.pool3(self.relu(self.conv3(x)))
#         x = self.pool4(self.relu(self.conv4(x)))
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = x.squeeze(0)
        return x

In [ ]:
torchsummary.summary(TumorClassifier(), (1, 224, 224))

In [ ]:
# Converting the Image data to dataloader form

dataloader_train = DataLoader(
    TRAIN_DATASET,
    shuffle=True,
    batch_size=1,
)

image, label = next(iter(dataloader_train))
print(image[0].squeeze().shape)

In [ ]:
image, label = next(iter(dataloader_train))
print(image.shape)

## Training

In [ ]:
net = TumorClassifier()

optimizer = torch.optim.SGD(net.parameters(), lr=0.02)

criterion = nn.MSELoss()

In [ ]:
torch.manual_seed(123)

In [ ]:
EPOCH = 30

net.train()
for i in range(EPOCH):
    epoch_loss = 0
    for data, label in tqdm(dataloader_train, total=len(dataloader_train)):
        
        # clearing all gradients
        optimizer.zero_grad()

        # getting outputs from data
        output = net(data)

        # getting the loss
        loss = criterion(output, label.float())

        # aggregating loss over an epoch
        epoch_loss += loss.item()
        
        # back propagation
        loss.backward()

        # gradient and weight updates
        optimizer.step()
    
    print(f"EPOCH: {i+1}/{EPOCH} -> Loss: {epoch_loss}")

## Validation

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Grayscale(),
])

VAL_DIR = '/kaggle/working/val/'
VAL_DATASET = ImageFolder(root=VAL_DIR, transform=test_transforms)

dataloader_val = DataLoader(
    VAL_DATASET,
    batch_size=1,
)

In [ ]:
net.eval()

metric = Accuracy(task='binary')

labels = []
predictions = []

with torch.no_grad():
    for data, label in dataloader_val:
        output = net.forward(data)
        prediction = round(output.numpy().flatten()[0])
#         print(f"Ground Truth Label: {label[0]}, Prediction: {prediction}")
        metric(output, label)
        labels.append(label)
        predictions.append(prediction)
    acc = metric.compute()
    
    print(f"Validation Accuracy: {acc*100:.2f}%")

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(confusion_matrix(labels, outputs), annot=True)

In [ ]:
metric = Accuracy(task='binary')

with torch.no_grad():
    for data, label in dataloader_train:
        output = net.forward(data)
        prediction = round(output.numpy().flatten()[0])
#         print(f"Ground Truth Label: {label[0]}, Prediction: {prediction}")
        metric(output, label)
    acc = metric.compute()
    
    print(f"Accuracy: {acc*100:.2f}%")

In [ ]:
torch.save(net.state_dict(), 'weights.pth')

In [ ]:
net = TumorClassifier()
net.load_state_dict(torch.load('weights.pth'))

In [ ]:
### test dataloader
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Grayscale(),
])

TEST_DIR = '/kaggle/working/test/'
TEST_DATASET = ImageFolder(root=TEST_DIR, transform=test_transforms)

dataloader_test = DataLoader(
    TEST_DATASET,
    batch_size=1,
)

In [ ]:
test_metric = Accuracy(task='binary')

net.eval()

with torch.no_grad():
    for data, label in dataloader_test:
        output = net.forward(data)
        prediction = round(output.numpy().flatten()[0])
        print(f"Ground Truth Label: {label[0]}, Prediction: {prediction}")
        test_metric(output, label)
        
    test_acc = test_metric.compute()
    print(f"Test Accuracy: {test_acc*100:.2f}%")